# Comparison Between TreeValue and Tianshou Batch

In this section, we will take a look at the feature and performance of the [Tianshou Batch](https://github.com/thu-ml/tianshou) library, which is developed by Tsinghua Machine Learning Group.

Before starting the comparison, let us define some thing.

In [1]:
import torch 

_TREE_DATA_1 = {'a': 1, 'b': 2, 'x': {'c': 3, 'd': 4}}
_TREE_DATA_2 = {
    'a': torch.randn(2, 3), 
    'x': {
        'c': torch.randn(3, 4)
    },
}
_TREE_DATA_3 = {
    'obs': torch.randn(4, 84, 84),
    'action': torch.randint(0, 6, size=(1,)),
    'reward': torch.rand(1),
}

/opt/hostedtoolcache/Python/3.8.16/x64/lib/python3.8/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


## Read and Write Operation

Reading and writing are the two most common operations in the tree data structure based on the data model (TreeValue and Tianshou Batch both belong to this type), so this section will compare the reading and writing performance of these two libraries.

### TreeValue's Get and Set

In [2]:
from treevalue import FastTreeValue

t = FastTreeValue(_TREE_DATA_2)

In [3]:
t

<FastTreeValue 0x7f7168437f70>
├── 'a' --> tensor([[ 1.2876,  1.9036,  0.7975],
│                   [ 0.7100, -0.9388,  0.4418]])
└── 'x' --> <FastTreeValue 0x7f7168437df0>
    └── 'c' --> tensor([[ 0.3508,  0.1516,  0.5272, -0.3053],
                        [ 0.2160,  1.0574, -1.5683,  1.0984],
                        [-0.2350,  0.1363,  0.7641,  1.0707]])

In [4]:
t.a

tensor([[ 1.2876,  1.9036,  0.7975],
        [ 0.7100, -0.9388,  0.4418]])

In [5]:
%timeit t.a

61.6 ns ± 0.0593 ns per loop (mean ± std. dev. of 7 runs, 10,000,000 loops each)


In [6]:
new_value = torch.randn(2, 3)
t.a = new_value

t

<FastTreeValue 0x7f7168437f70>
├── 'a' --> tensor([[ 0.2611,  0.4028, -0.2588],
│                   [ 0.0465, -0.1334, -0.2851]])
└── 'x' --> <FastTreeValue 0x7f7168437df0>
    └── 'c' --> tensor([[ 0.3508,  0.1516,  0.5272, -0.3053],
                        [ 0.2160,  1.0574, -1.5683,  1.0984],
                        [-0.2350,  0.1363,  0.7641,  1.0707]])

In [7]:
%timeit t.a = new_value

64.5 ns ± 0.0792 ns per loop (mean ± std. dev. of 7 runs, 10,000,000 loops each)


### Tianshou Batch's Get and Set

In [8]:
from tianshou.data import Batch

b = Batch(**_TREE_DATA_2)

In [9]:
b

Batch(
    a: tensor([[ 1.2876,  1.9036,  0.7975],
               [ 0.7100, -0.9388,  0.4418]]),
    x: Batch(
           c: tensor([[ 0.3508,  0.1516,  0.5272, -0.3053],
                      [ 0.2160,  1.0574, -1.5683,  1.0984],
                      [-0.2350,  0.1363,  0.7641,  1.0707]]),
       ),
)

In [10]:
b.a

tensor([[ 1.2876,  1.9036,  0.7975],
        [ 0.7100, -0.9388,  0.4418]])

In [11]:
%timeit b.a

61.5 ns ± 0.0272 ns per loop (mean ± std. dev. of 7 runs, 10,000,000 loops each)


In [12]:
new_value = torch.randn(2, 3)
b.a = new_value

b

Batch(
    a: tensor([[ 1.4218e+00,  1.0150e-01,  1.0111e+00],
               [-6.9108e-01, -3.8175e-04,  1.3663e-01]]),
    x: Batch(
           c: tensor([[ 0.3508,  0.1516,  0.5272, -0.3053],
                      [ 0.2160,  1.0574, -1.5683,  1.0984],
                      [-0.2350,  0.1363,  0.7641,  1.0707]]),
       ),
)

In [13]:
%timeit b.a = new_value

490 ns ± 0.0707 ns per loop (mean ± std. dev. of 7 runs, 1,000,000 loops each)


## Initialization

### TreeValue's Initialization

In [14]:
%timeit FastTreeValue(_TREE_DATA_1)

828 ns ± 0.119 ns per loop (mean ± std. dev. of 7 runs, 1,000,000 loops each)


### Tianshou Batch's Initialization

In [15]:
%timeit Batch(**_TREE_DATA_1)

11.9 µs ± 73.3 ns per loop (mean ± std. dev. of 7 runs, 100,000 loops each)


## Deep Copy Operation

In [16]:
import copy

### Deep Copy of TreeValue

In [17]:
t3 = FastTreeValue(_TREE_DATA_3)
%timeit copy.deepcopy(t3)

138 µs ± 249 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


### Deep Copy of Tianshou Batch

In [18]:
b3 = Batch(**_TREE_DATA_3)
%timeit copy.deepcopy(b3)

139 µs ± 914 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


## Stack, Concat and Split Operation

### Performance of TreeValue

In [19]:
trees = [FastTreeValue(_TREE_DATA_2) for _ in range(8)]

In [20]:
t_stack = FastTreeValue.func(subside=True)(torch.stack)

t_stack(trees)

<FastTreeValue 0x7f70b3661dc0>
├── 'a' --> tensor([[[ 1.2876,  1.9036,  0.7975],
│                    [ 0.7100, -0.9388,  0.4418]],
│           
│                   [[ 1.2876,  1.9036,  0.7975],
│                    [ 0.7100, -0.9388,  0.4418]],
│           
│                   [[ 1.2876,  1.9036,  0.7975],
│                    [ 0.7100, -0.9388,  0.4418]],
│           
│                   [[ 1.2876,  1.9036,  0.7975],
│                    [ 0.7100, -0.9388,  0.4418]],
│           
│                   [[ 1.2876,  1.9036,  0.7975],
│                    [ 0.7100, -0.9388,  0.4418]],
│           
│                   [[ 1.2876,  1.9036,  0.7975],
│                    [ 0.7100, -0.9388,  0.4418]],
│           
│                   [[ 1.2876,  1.9036,  0.7975],
│                    [ 0.7100, -0.9388,  0.4418]],
│           
│                   [[ 1.2876,  1.9036,  0.7975],
│                    [ 0.7100, -0.9388,  0.4418]]])
└── 'x' --> <FastTreeValue 0x7f70b3661c10>
    └── 'c' --> tensor([[[

In [21]:
%timeit t_stack(trees)

32.9 µs ± 28.8 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [22]:
t_cat = FastTreeValue.func(subside=True)(torch.cat)

t_cat(trees)

<FastTreeValue 0x7f70b3cabc70>
├── 'a' --> tensor([[ 1.2876,  1.9036,  0.7975],
│                   [ 0.7100, -0.9388,  0.4418],
│                   [ 1.2876,  1.9036,  0.7975],
│                   [ 0.7100, -0.9388,  0.4418],
│                   [ 1.2876,  1.9036,  0.7975],
│                   [ 0.7100, -0.9388,  0.4418],
│                   [ 1.2876,  1.9036,  0.7975],
│                   [ 0.7100, -0.9388,  0.4418],
│                   [ 1.2876,  1.9036,  0.7975],
│                   [ 0.7100, -0.9388,  0.4418],
│                   [ 1.2876,  1.9036,  0.7975],
│                   [ 0.7100, -0.9388,  0.4418],
│                   [ 1.2876,  1.9036,  0.7975],
│                   [ 0.7100, -0.9388,  0.4418],
│                   [ 1.2876,  1.9036,  0.7975],
│                   [ 0.7100, -0.9388,  0.4418]])
└── 'x' --> <FastTreeValue 0x7f71683d86a0>
    └── 'c' --> tensor([[ 0.3508,  0.1516,  0.5272, -0.3053],
                        [ 0.2160,  1.0574, -1.5683,  1.0984],
                 

In [23]:
%timeit t_cat(trees)

30.8 µs ± 49.6 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [24]:
t_split = FastTreeValue.func(rise=True)(torch.split)
tree = FastTreeValue({
    'obs': torch.randn(8, 4, 84, 84),
    'action': torch.randint(0, 6, size=(8, 1,)),
    'reward': torch.rand(8, 1),
})

%timeit t_split(tree, 1)

76.6 µs ± 260 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


### Performance of Tianshou Batch

In [25]:
batches = [Batch(**_TREE_DATA_2) for _ in range(8)]

Batch.stack(batches)

Batch(
    a: tensor([[[ 1.2876,  1.9036,  0.7975],
                [ 0.7100, -0.9388,  0.4418]],
       
               [[ 1.2876,  1.9036,  0.7975],
                [ 0.7100, -0.9388,  0.4418]],
       
               [[ 1.2876,  1.9036,  0.7975],
                [ 0.7100, -0.9388,  0.4418]],
       
               [[ 1.2876,  1.9036,  0.7975],
                [ 0.7100, -0.9388,  0.4418]],
       
               [[ 1.2876,  1.9036,  0.7975],
                [ 0.7100, -0.9388,  0.4418]],
       
               [[ 1.2876,  1.9036,  0.7975],
                [ 0.7100, -0.9388,  0.4418]],
       
               [[ 1.2876,  1.9036,  0.7975],
                [ 0.7100, -0.9388,  0.4418]],
       
               [[ 1.2876,  1.9036,  0.7975],
                [ 0.7100, -0.9388,  0.4418]]]),
    x: Batch(
           c: tensor([[[ 0.3508,  0.1516,  0.5272, -0.3053],
                       [ 0.2160,  1.0574, -1.5683,  1.0984],
                       [-0.2350,  0.1363,  0.7641,  1.0707]],
         

In [26]:
%timeit Batch.stack(batches)

79 µs ± 146 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [27]:
Batch.cat(batches)

Batch(
    a: tensor([[ 1.2876,  1.9036,  0.7975],
               [ 0.7100, -0.9388,  0.4418],
               [ 1.2876,  1.9036,  0.7975],
               [ 0.7100, -0.9388,  0.4418],
               [ 1.2876,  1.9036,  0.7975],
               [ 0.7100, -0.9388,  0.4418],
               [ 1.2876,  1.9036,  0.7975],
               [ 0.7100, -0.9388,  0.4418],
               [ 1.2876,  1.9036,  0.7975],
               [ 0.7100, -0.9388,  0.4418],
               [ 1.2876,  1.9036,  0.7975],
               [ 0.7100, -0.9388,  0.4418],
               [ 1.2876,  1.9036,  0.7975],
               [ 0.7100, -0.9388,  0.4418],
               [ 1.2876,  1.9036,  0.7975],
               [ 0.7100, -0.9388,  0.4418]]),
    x: Batch(
           c: tensor([[ 0.3508,  0.1516,  0.5272, -0.3053],
                      [ 0.2160,  1.0574, -1.5683,  1.0984],
                      [-0.2350,  0.1363,  0.7641,  1.0707],
                      [ 0.3508,  0.1516,  0.5272, -0.3053],
                      [ 0.2160,  

In [28]:
%timeit Batch.cat(batches)

145 µs ± 219 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [29]:
batch = Batch({
    'obs': torch.randn(8, 4, 84, 84),
    'action': torch.randint(0, 6, size=(8, 1,)),
    'reward': torch.rand(8, 1)}
)

%timeit list(Batch.split(batch, 1, shuffle=False, merge_last=True))

306 µs ± 490 ns per loop (mean ± std. dev. of 7 runs, 1,000 loops each)
